In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

import requests 
from urllib.parse import urlencode 
from tqdm.auto import tqdm
from scipy import stats

plt.style.use('ggplot')


In [3]:
users_test = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/4XXIME4osGrMRA', sep=';')
users_control_1 = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/yJFydMNNGkEKfg', sep=';')
users_control_2 = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/br6KkQupzzTGoQ', sep=';')
transactions_test = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/gvCWpZ55ODzs2g', sep=';')
transactions_control_1 = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/VY5W0keMX5TZBQ', sep=';') 
transactions_control_2 = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/th5GL0mGOc-qzg', sep=';')

In [4]:
#или можно загрузить файлы локально
#users_test = pd.read_csv('Проект_3_users_test.csv', sep=';')
#users_control_1 = pd.read_csv('Проект_3_users_control_1.csv', sep=';')
#users_control_2 = pd.read_csv('Проект_3_users_control_2.csv', sep=';')
#transactions_test = pd.read_csv('Проект_3_transactions_test.csv', sep=';')
#transactions_control_1 = pd.read_csv('Проект_3_transactions_control_1.csv', sep=';')  
#transactions_control_2 = pd.read_csv('Проект_3_transactions_control_2.csv', sep=';')

In [5]:
#посмотрим на данные, начнем с данных о пользователях тестовой группы
users_test.head()

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892309896,27,685,NaN,United States of America,"1,2,3,4,5,6,7,8,9,10,11,12,16,17",1,24,30,89,NaN,NaN,0
1,892044516,27,0,NaN,Germany,NaN,1,24,30,0,NaN,NaN,0
2,892185708,45,44,NaN,Israel,"1,3,4,6,7,8,10,11,12,13,14,15,16,17,18,19,20,2...",1,42,48,68,NaN,NaN,0
3,892130292,32,0,NaN,United States of America,"1,2,3,4,5,6,7,8,9,10,11,12",1,29,35,0,NaN,NaN,0
4,891406035,27,1000,NaN,France,NaN,1,24,30,1,1.0,NaN,0


In [6]:
#посмотрим на типы данных
users_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308 entries, 0 to 4307
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uid               4308 non-null   int64  
 1   age               4308 non-null   int64  
 2   attraction_coeff  4308 non-null   int64  
 3   coins             49 non-null     float64
 4   country           4308 non-null   object 
 5   visit_days        2978 non-null   object 
 6   gender            4308 non-null   int64  
 7   age_filter_start  4308 non-null   int64  
 8   age_filter_end    4308 non-null   int64  
 9   views_count       4308 non-null   int64  
 10  was_premium       408 non-null    float64
 11  is_premium        157 non-null    float64
 12  total_revenue     4308 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 437.7+ KB


In [7]:
#проверим данные на наличие пропущенных значений
users_test.isna().sum()

uid                    0
age                    0
attraction_coeff       0
coins               4259
country                0
visit_days          1330
gender                 0
age_filter_start       0
age_filter_end         0
views_count            0
was_premium         3900
is_premium          4151
total_revenue          0
dtype: int64

In [8]:
#пропущенные значения имеются в 4-х колонках и выглядят допустимыми. заменим их на 0 для удобства.
users_test = users_test.fillna(0)

In [9]:
#проверим данные на наличие дубликатов
users_test.duplicated().sum()

0

In [10]:
#исследуем данные о транзакциях тестовой группы
transactions_test.head()

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891345942,Italy,2017-05-11 13:00:00,2017-11-13 15:04:00,12909,147,trial_vip_popup,trial_premium
1,892054251,United States of America,2017-10-22 00:33:00,2017-10-30 01:37:00,13923,147,trial_vip_popup,trial_premium
2,892236423,United States of America,2017-10-18 01:09:00,2017-10-23 00:15:00,3783,67,menu,other_type
3,892236423,United States of America,2017-10-18 01:09:00,2017-10-27 22:38:00,3783,67,menu,other_type
4,892168170,United States of America,2017-10-19 17:10:00,2017-10-27 19:10:00,9087,147,trial_vip_popup,trial_premium


In [11]:
#посмотрим на типы данных
transactions_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uid           273 non-null    int64 
 1   country       273 non-null    object
 2   joined_at     273 non-null    object
 3   paid_at       273 non-null    object
 4   revenue       273 non-null    int64 
 5   payment_id    273 non-null    int64 
 6   from_page     273 non-null    object
 7   product_type  273 non-null    object
dtypes: int64(3), object(5)
memory usage: 17.2+ KB


In [12]:
#изменим типы данных колонок, содержащих даты

transactions_test[['joined_at', 
        'paid_at']] = transactions_test[['joined_at', 
        'paid_at']].apply(pd.to_datetime)

In [13]:
transactions_test.dtypes
#теперь ок

uid                      int64
country                 object
joined_at       datetime64[ns]
paid_at         datetime64[ns]
revenue                  int64
payment_id               int64
from_page               object
product_type            object
dtype: object

In [14]:
#проверим данные на наличие пропущенных значений
transactions_test.isna().sum()

uid             0
country         0
joined_at       0
paid_at         0
revenue         0
payment_id      0
from_page       0
product_type    0
dtype: int64

In [15]:
#проверим данные на наличие дубликатов
transactions_test.duplicated().sum()

7

In [16]:
#избавимся от дубликатов
transactions_test = transactions_test.drop_duplicates()

In [17]:
#исследуем данные о пользователях 1-ой контрольной группы
users_control_1.head()

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892319115,25,435,NaN,United States of America,"1,2,3,4,5,6",0,22,32,982,NaN,NaN,0
1,891248523,29,500,NaN,United States of America,"1,2",1,26,32,12,NaN,NaN,0
2,891670932,33,800,NaN,France,"1,2,3,5,6,8,9,11,13,16",1,30,36,5,NaN,NaN,0
3,891060786,26,0,NaN,Argentina,"1,2,3,4,5",1,23,29,0,NaN,NaN,0
4,892006554,35,294,NaN,United States of America,"1,2,3,5,6,7,8,9,10,12,15,16,17,19",1,30,40,17,NaN,NaN,0


In [18]:
users_control_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uid               4340 non-null   int64  
 1   age               4340 non-null   int64  
 2   attraction_coeff  4340 non-null   int64  
 3   coins             60 non-null     float64
 4   country           4340 non-null   object 
 5   visit_days        3016 non-null   object 
 6   gender            4340 non-null   int64  
 7   age_filter_start  4340 non-null   int64  
 8   age_filter_end    4340 non-null   int64  
 9   views_count       4340 non-null   int64  
 10  was_premium       436 non-null    float64
 11  is_premium        192 non-null    float64
 12  total_revenue     4340 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 440.9+ KB


In [19]:
#проверим данные на наличие пропущенных значений
users_control_1.isna().sum()

uid                    0
age                    0
attraction_coeff       0
coins               4280
country                0
visit_days          1324
gender                 0
age_filter_start       0
age_filter_end         0
views_count            0
was_premium         3904
is_premium          4148
total_revenue          0
dtype: int64

In [20]:
#пропущенные значения как и в тестовой группе имеются в 4-х колонках и выглядят допустимыми. 
#заменим их на 0 для удобства.
users_control_1 = users_control_1.fillna(0)

In [21]:
#проверим данные на наличие дубликатов
users_control_1.duplicated().sum()

0

In [22]:
#исследуем данные о транзакциях 1-ой контрольной группы
transactions_control_1.head()

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891319275.0,France,2017-05-11 19:57:00,2017-12-11 21:57:00,12727.0,147.0,trial_vip_popup,trial_premium
1,892421826.0,Israel,2017-10-14 12:46:00,2017-10-23 12:54:00,14586.0,147.0,trial_vip_popup,trial_premium
2,891822480.0,Italy,2017-10-26 22:27:00,2017-12-11 20:59:00,1911.0,19.0,none,coins
3,891367521.0,Italy,2017-05-11 02:37:00,2017-06-11 19:15:00,1456.0,127.0,sympathy,other_type
4,892207959.0,France,2017-10-18 18:30:00,2017-12-11 12:03:00,1261.0,19.0,empty_likes,coins


In [23]:
transactions_control_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uid           377 non-null    float64
 1   country       377 non-null    object 
 2   joined_at     1007 non-null   object 
 3   paid_at       1007 non-null   object 
 4   revenue       377 non-null    float64
 5   payment_id    377 non-null    float64
 6   from_page     377 non-null    object 
 7   product_type  377 non-null    object 
dtypes: float64(3), object(5)
memory usage: 63.1+ KB


In [24]:
#изменим типы данных колонок, содержащих даты

transactions_control_1[['joined_at', 
        'paid_at']] = transactions_control_1[['joined_at', 
        'paid_at']].apply(pd.to_datetime)

#и приведем колонки uid,revenue, payment_id к типу int64 
#transactions_control_1.uid = transactions_control_1.uid.astype('int64')
#transactions_control_1.revenue = transactions_control_1.revenue.astype('int64')
#transactions_control_1.payment_id = transactions_control_1.payment_id.astype('int64')


In [25]:
transactions_control_1.dtypes
#теперь ок

uid                    float64
country                 object
joined_at       datetime64[ns]
paid_at         datetime64[ns]
revenue                float64
payment_id             float64
from_page               object
product_type            object
dtype: object

In [26]:
#проверим данные на наличие пропущенных значений
transactions_control_1.isna().sum()

uid             630
country         630
joined_at       630
paid_at         630
revenue         630
payment_id      630
from_page       630
product_type    630
dtype: int64

In [27]:
#избавимся от строк, в которых пропущены значения.

transactions_control_1 = transactions_control_1.dropna()

In [28]:
#проверим данные на наличие дубликатов
transactions_control_1.duplicated().sum()

21

In [29]:
#избавимся от дубликатов
transactions_control_1 = transactions_control_1.drop_duplicates()

In [30]:
#исследуем данные о пользователях 2-ой контрольной группы
users_control_2.head()

,uid,age,attraction_coeff,coins,country,visit_days,gender,age_filter_start,age_filter_end,views_count,was_premium,is_premium,total_revenue
0,892035504,37,137,NaN,Israel,"1,2,3,4,5,6,7,8,9,11,13",1,30,40,51,NaN,NaN,0
1,891782112,57,0,NaN,Italy,1,1,54,60,0,NaN,NaN,0
2,891110337,30,769,NaN,France,"1,2,3,4,5",1,27,33,13,NaN,NaN,0
3,891796320,48,750,NaN,France,"1,4,6,8,10,11,12,14,15,16,18",1,45,51,12,NaN,NaN,0
4,891880212,54,638,NaN,United States of America,1,1,35,53,94,NaN,NaN,0


In [31]:
users_control_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4264 entries, 0 to 4263
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   uid               4264 non-null   int64  
 1   age               4264 non-null   int64  
 2   attraction_coeff  4264 non-null   int64  
 3   coins             61 non-null     float64
 4   country           4264 non-null   object 
 5   visit_days        2900 non-null   object 
 6   gender            4264 non-null   int64  
 7   age_filter_start  4264 non-null   int64  
 8   age_filter_end    4264 non-null   int64  
 9   views_count       4264 non-null   int64  
 10  was_premium       411 non-null    float64
 11  is_premium        191 non-null    float64
 12  total_revenue     4264 non-null   int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 433.2+ KB


In [32]:
#проверим данные на наличие пропущенных значений
users_control_2.isna().sum()

uid                    0
age                    0
attraction_coeff       0
coins               4203
country                0
visit_days          1364
gender                 0
age_filter_start       0
age_filter_end         0
views_count            0
was_premium         3853
is_premium          4073
total_revenue          0
dtype: int64

In [33]:
#пропущенные значения как и в тестовой и 1-контрольной имеются в 4-х колонках и выглядят допустимыми. 
#заменим их на 0 для удобства.
users_control_2 = users_control_2.fillna(0)

In [34]:
#проверим данные на наличие дубликатов
users_control_2.duplicated().sum()

0

In [35]:
#исследуем данные о транзакциях 2-ой контрольной группы
transactions_control_2.head()

,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
0,891266616,Argentina,2017-06-11 15:25:00,2017-10-11 17:35:00,6305,19,autorefill,coins
1,892186737,Chile,2017-10-19 06:03:00,2017-04-11 08:53:00,4732,147,promo_09,premium_no_trial
2,891959004,United States of America,2017-10-24 00:12:00,2017-10-31 02:12:00,10153,147,trial_vip_popup,trial_premium
3,892115478,Spain,2017-10-20 20:39:00,2017-10-26 06:26:00,2366,67,empty_likes,other_type
4,891592941,Switzerland,2017-10-31 12:40:00,2017-03-11 16:25:00,6292,147,promo_09,premium_no_trial


In [36]:
transactions_control_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uid           328 non-null    int64 
 1   country       328 non-null    object
 2   joined_at     328 non-null    object
 3   paid_at       328 non-null    object
 4   revenue       328 non-null    int64 
 5   payment_id    328 non-null    int64 
 6   from_page     328 non-null    object
 7   product_type  328 non-null    object
dtypes: int64(3), object(5)
memory usage: 20.6+ KB


In [37]:
#изменим типы данных колонок, содержащих даты

transactions_control_2[['joined_at', 
        'paid_at']] = transactions_control_2[['joined_at', 
        'paid_at']].apply(pd.to_datetime)

In [38]:
transactions_control_1.dtypes
#теперь ок

uid                    float64
country                 object
joined_at       datetime64[ns]
paid_at         datetime64[ns]
revenue                float64
payment_id             float64
from_page               object
product_type            object
dtype: object

In [39]:
#проверим данные на наличие пропущенных значений
transactions_control_2.isna().sum()

uid             0
country         0
joined_at       0
paid_at         0
revenue         0
payment_id      0
from_page       0
product_type    0
dtype: int64

In [40]:
#проверим данные на наличие дубликатов
transactions_control_2.duplicated().sum()

5

In [41]:
#избавимся от дубликатов
transactions_control_2 = transactions_control_2.drop_duplicates()

In [42]:
#проверим данные о транзакциях пользователей
print("transactions_test size:      {}".format(transactions_test.shape))
print("transactions_control_1 size: {}".format(transactions_control_1.shape))
print("transactions_control_2 size: {}".format(transactions_control_2.shape))

transactions_test size:      (266, 8)
transactions_control_1 size: (356, 8)
transactions_control_2 size: (323, 8)


In [43]:
#поскольку размер датасетов отличается, сверим даты проведения эксперимента
print('Dataset "transactions_test"')
print('Begin date: {}'.format(transactions_test.paid_at.min().strftime(format='%Y-%m-%d')))
print('End date: {}'.format(transactions_test.paid_at.max().strftime(format='%Y-%m-%d')))

Dataset "transactions_test"
Begin date: 2017-01-11
End date: 2017-12-11


In [44]:
print('Dataset "transactions_control_1"')
print('Begin date: {}'.format(transactions_control_1.paid_at.min().strftime(format='%Y-%m-%d')))
print('End date: {}'.format(transactions_control_1.paid_at.max().strftime(format='%Y-%m-%d')))

Dataset "transactions_control_1"
Begin date: 2016-12-14
End date: 2017-12-11


In [45]:
print('Dataset "transactions_control_2"')
print('Begin date: {}'.format(transactions_control_2.paid_at.min().strftime(format='%Y-%m-%d')))
print('End date: {}'.format(transactions_control_2.paid_at.max().strftime(format='%Y-%m-%d')))

Dataset "transactions_control_2"
Begin date: 2017-01-11
End date: 2017-12-11


In [46]:
#видим, что у контрольной 1 группы отличается дата начала эксперимента, уберем эти данные
transactions_control_1 = transactions_control_1[transactions_control_1.paid_at >= '2017-02-11']

In [47]:
#проверим, у всех ли пользователей корректные данные по регистрации и покупке
print(transactions_test[transactions_test.joined_at > transactions_test.paid_at].shape)
transactions_test[transactions_test.joined_at > transactions_test.paid_at].head()

(47, 8)


,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
11,891861048,Canada,2017-10-26 02:55:00,2017-10-11 19:00:00,3588,68,none,premium_no_trial
20,891806280,United States of America,2017-10-27 08:56:00,2017-03-11 10:57:00,8242,147,trial_vip_popup,trial_premium
24,891786216,Spain,2017-10-27 18:20:00,2017-03-11 20:20:00,12818,147,trial_vip_popup,trial_premium
25,892057347,France,2017-10-21 23:15:00,2017-08-11 01:56:00,1209,19,none,coins
36,891589239,Italy,2017-10-31 14:16:00,2017-09-11 15:44:00,6305,19,empty_likes,coins


In [48]:
print(transactions_control_1[transactions_control_1.joined_at > transactions_control_1.paid_at].shape)
transactions_control_1[transactions_control_1.joined_at > transactions_control_1.paid_at].head()

(48, 8)


,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
5,891721851.0,United States of America,2017-10-29 02:50:00,2017-05-11 07:15:00,4602.0,146.0,empty_likes,coins
14,892277877.0,United States of America,2017-10-17 03:36:00,2017-10-11 15:23:00,897.0,19.0,empty_likes,coins
20,892366260.0,United Kingdom (Great Britain),2017-10-15 15:04:00,2017-09-11 20:06:00,6305.0,19.0,autorefill,coins
21,891707061.0,Argentina,2017-10-29 10:54:00,2017-04-11 00:30:00,1261.0,19.0,empty_likes,coins
23,891707061.0,Argentina,2017-10-29 10:54:00,2017-10-11 00:04:00,1261.0,19.0,autorefill,coins


In [49]:
print(transactions_control_2[transactions_control_2.joined_at > transactions_control_2.paid_at].shape)
transactions_control_2[transactions_control_2.joined_at > transactions_control_2.paid_at].head()

(60, 8)


,uid,country,joined_at,paid_at,revenue,payment_id,from_page,product_type
1,892186737,Chile,2017-10-19 06:03:00,2017-04-11 08:53:00,4732,147,promo_09,premium_no_trial
4,891592941,Switzerland,2017-10-31 12:40:00,2017-03-11 16:25:00,6292,147,promo_09,premium_no_trial
14,891655107,United States of America,2017-10-30 05:06:00,2017-08-11 19:12:00,9087,147,trial_vip_popup,trial_premium
19,891606141,United States of America,2017-10-31 04:43:00,2017-03-11 21:14:00,3783,67,empty_likes,other_type
20,891609360,Italy,2017-10-31 02:07:00,2017-01-11 23:19:00,3913,146,empty_likes,coins


Обнаружено достаточно большое количество пользователей, у которых дата регистрации позже чем дата оплаты. Предположим, что это ошибка при записи данных. При этом даты транзакций и даты регистрации пользователей попадают в период проведения эксперимента, в связи с чем кажется логичным использовать их в нашем исследовании. 

## Результаты:
Предварительный разведывательный анализ данных закончен.  
Мы почистили данные (удалили дубликаты и данные не подходящие нам по дате, заполнили пропуски) и приняли решение использовать данные пользователей, у которых дата регистрации позже чем дата оплаты.  
Теперь можем приступать к исследованию успешности проведения эксперимента по изменению стоимости премиум-подписки.

Нашими ключевыми метриками будут средняя выручка с пользователя (ARPU) и конверсия в премиум-подписку (CR).   
Сначала мы проведем А/А-тесты, чтобы убедиться, что система сплитования сработала корректно.

In [50]:
#для удобства соединим датасеты с данными о пользователях с датасетами о транзакциях
test_group = users_test.merge(transactions_test, on = 'uid', how = 'left')
control_group_1 = users_control_1.merge(transactions_control_1, on = 'uid', how ='left')
control_group_2 = users_control_2.merge(transactions_control_2, on = 'uid', how ='left')


In [51]:
#начнем с А/А теста по ARPU

from scipy import stats
from tqdm.auto import tqdm

control_revenue_1 = control_group_1.groupby('uid',as_index=False).agg({'revenue':'sum'})
control_revenue_2 = control_group_2.groupby('uid',as_index=False).agg({'revenue':'sum'})
n = 1000
simulations = 1000
n_s = 300
result = []

#запускаем симуляцию
for i in tqdm(range(simulations)):
    s1 = control_revenue_1.revenue.sample(n_s, replace=False).values
    s2 = control_revenue_2.revenue.sample(n_s, replace=False).values
    result.append(stats.ttest_ind(s1, s2, equal_var = False)[1]) 

# проверим, что количество ложноположительных случаев не превышает нашу альфа
sum(np.array(result) < 0.05) / simulations


0.029

Полученный результат < 0.05, значит мы можем сделать вывод, что сплитование сработало корректно.

In [52]:
#теперь проведем А/А тест по CR, воспользовавшись критерием хи-квадрат
control_group_1['premium_no_trial'] = np.where(control_group_1.product_type == 'premium_no_trial', 1, 0)
control_group_2['premium_no_trial'] = np.where(control_group_2.product_type == 'premium_no_trial', 1, 0)
test_group['premium_no_trial'] = np.where(test_group.product_type == 'premium_no_trial', 1, 0)


n = 10000
simulations = 1000
n_s = 1000
result = []

# Запуск симуляций A/A теста
for i in tqdm(range(simulations)):
    s1 = control_group_1.premium_no_trial.sample(n_s, replace=False).values
    s2 = control_group_2.premium_no_trial.sample(n_s, replace=False).values
    ab = np.array([[s1.sum(), s1.shape[0] - s1.sum()], [s2.sum(), s2.shape[0] - s2.sum()]])  
    result.append(stats.chi2_contingency(ab, correction=False)[1])

# проверим, что количество ложноположительных случаев не превышает нашу альфу
sum(np.array(result) < 0.05) / simulations

0.025

## Результаты:
Полученный результат также < 0.05, значит мы можем сделать вывод, что сплитование сработало корректно.

In [53]:
#можно приступать к A/B тесту
#сначала для удобства объединим наши две контрольные группы
control_group = pd.concat([control_group_1,control_group_2])

In [54]:
#чтобы иметь общую картину посмотрим на показатели по выручке у пользователей с премиум подпиской в тестовой группе
test_group[test_group.product_type == 'premium_no_trial'].revenue.describe()

count        76.000000
mean      10667.184211
std       17307.705262
min        1898.000000
25%        6012.500000
50%        8021.000000
75%       10052.250000
max      113477.000000
Name: revenue, dtype: float64

In [55]:
#а также на показатели по выручке у пользователей с премиум подпиской в контрольной группе
control_group[control_group.product_type == 'premium_no_trial'].revenue.describe()

count      211.00000
mean      7179.57346
std       9933.41099
min       1482.00000
25%       4537.00000
50%       6292.00000
75%       6292.00000
max      81796.00000
Name: revenue, dtype: float64

Показатели в тестовой группе выше, теперь необходимо выяснить статистически значимы ли эти различия.

In [56]:
#сначала проверим ARPU на нормальность распределения

#для тестовой группы
print('Test group: ', 'stats:',
      stats.shapiro(test_group.query('revenue > 0').revenue)[0],
      '  pvalue:',
      stats.shapiro(test_group.query('revenue > 0').revenue)[1])

#для контрольной группы
print('Control group: ', 'stats:',
      stats.shapiro(control_group.query('revenue > 0').revenue)[0],
      '  pvalue:',
      stats.shapiro(control_group.query('revenue > 0').revenue)[1])



Test group:  stats: 0.49752235412597656   pvalue: 8.011712953511395e-27
Control group:  stats: 0.5692328214645386   pvalue: 1.6653131843525557e-37


Поскольку p < 0.05 мы можем сделать вывод о ненормальном распределении данных по выручке, что не является фактором, препятствующему проведению ttest :)


In [57]:
#проверим, есть ли статистическая значимость по метрике ARPU, используя ttest

In [58]:
test_group_ARPU = test_group \
    .groupby('uid', as_index=False) \
    .agg({'revenue': 'sum'}) 

control_group_ARPU = control_group \
    .groupby('uid', as_index=False) \
    .agg({'revenue': 'sum'}) 
stats.ttest_ind(test_group_ARPU.revenue, control_group_ARPU.revenue).pvalue

0.6987666612446526

Исходя из результата ttest мы можем сделать вывод, что между группами нет статистической значимости в средних значениях ARPU.

In [59]:
#теперь проверим, что с метрикой CR (категориальная переменная)

In [60]:
print('test_group_CR = ', round(((test_group.query('revenue > 0') \
                                  .uid.nunique()\
                                  / test_group.uid.nunique()) * 100),0), '%')

test_group_CR =  3.0 %


In [61]:
print('control_group_CR = ', round(((control_group.query('revenue > 0') \
                                  .uid.nunique()\
                                  / control_group.uid.nunique()) * 100),0), '%')

control_group_CR =  4.0 %


In [62]:
#на первый взгляд конверсия в тестовой группе ниже; теперь проверим статзначимость, используя критерий хи-квадрат
control_group_CR = control_group.premium_no_trial
test_group_CR = test_group.premium_no_trial
res = np.array([[control_group_CR.sum(), control_group_CR.shape[0] - control_group_CR.sum()], [test_group_CR.sum(), test_group_CR.shape[0] - test_group_CR.sum()]])
stats.chi2_contingency(res, correction=False)[1]

0.014085963830302464

Исходя из результатов теста хи-квадрат мы можем сделать вывод, что конверсия в премиум-подписку тестовой группы статзначимо снизилась по сравнению с контрольной группой.

## Результаты:
Проанализировав итоги эксперимента, мы можем сделать следующие выводы:

- эксперимент проведен корректно (с системой сплитования все ок);
- при этом по результатам эксперимента APRU не изменилась, а CR в тестовой группе снизилась, в связи с чем в целом наш эксперимент не является успешным.   
